In [2]:
import datetime as dt
import pandas as pd

from de.processor.dart import DART_FUNDAMENTAL_PROCESSOR, DART_INFO_PROCESSOR
from de.processor.krx import KRX_INFO_PROCESSOR
from de.processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR

In [3]:
stockcodes = {'002450',
 '007720',
 '014190',
 '035420',
 '044780',
 '067900',
 '073560',
 '221980'}

In [4]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [5]:
# dart_info
dart_info_df = pd.read_csv("./de/data/dart_info_df.csv", index_col=0)
dart_info_processor = DART_INFO_PROCESSOR(dart_info_df)

In [6]:
# krx_info
krx_info_df = pd.read_csv("./de/data/krx_info_df.csv", index_col=0)
krx_info_processor = KRX_INFO_PROCESSOR(krx_info_df)

In [7]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [8]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [9]:
class STOCK_INFO_PROCESOR:
    def __init__(
        self, fdr_info_processor, krx_info_processor, dart_info_processor, dart_fundamental_processor, ohlcv_df
    ):
        self.fdr_info_processor = fdr_info_processor
        self.krx_info_processor = krx_info_processor
        self.dart_info_processor = dart_info_processor
        self.dart_fundamental_processor = dart_fundamental_processor
        self.ohlcv_df = ohlcv_df

    @staticmethod
    def append_StockName(df, fdr_info_processor):
        df["StockName"] = df["StockCode"].map(fdr_info_processor.get_StockName_dict())
        return df

    @staticmethod
    def append_MarketSector(df, krx_info_processor):
        df["MarketSector"] = df["StockCode"].map(krx_info_processor.get_MarketSector_dict())
        return df

    @staticmethod
    def append_Product(df, dart_info_processor):
        df["Product"] = df["StockCode"].map(dart_info_processor.get_Product_dict())
        return df

    @staticmethod
    def append_bps(df, dart_fundamental_processor, fdr_info_processor):
        df["BPS"] = (
            df["StockCode"].map(dart_fundamental_processor.get_TotalAssets_dict())
            / df["StockCode"].map(fdr_info_processor.get_Shares_dict())
        ).astype(int)
        return df

    @staticmethod
    def append_eps(df, dart_fundamental_processor, fdr_info_processor):
        df["EPS"] = (
            df["StockCode"].map(dart_fundamental_processor.get_NetProfit_dict())
            / df["StockCode"].map(fdr_info_processor.get_Shares_dict())
        ).astype(int)
        return df

    @staticmethod
    def append_price(df, ohlcv_df):
        price_dict = ohlcv_df[ohlcv_df["Date"] == ohlcv_df["Date"].max()].set_index("StockCode")["Close"].to_dict()
        df["Price"] = df["StockCode"].map(price_dict)
        return df

    def get_stock_info_df(self, stockcodes):
        stock_info_df = pd.DataFrame(stockcodes, columns=["StockCode"])
        stock_info_df = self.append_StockName(stock_info_df, self.fdr_info_processor)
        stock_info_df = self.append_MarketSector(stock_info_df, self.krx_info_processor)
        stock_info_df = self.append_Product(stock_info_df, self.dart_info_processor)
        stock_info_df = self.append_eps(stock_info_df, self.dart_fundamental_processor, self.fdr_info_processor)
        stock_info_df = self.append_bps(stock_info_df, self.dart_fundamental_processor, self.fdr_info_processor)
        stock_info_df = self.append_price(stock_info_df, self.ohlcv_df)
        return stock_info_df

In [10]:
stock_info_procesor = STOCK_INFO_PROCESOR(
    fdr_info_processor, krx_info_processor, dart_info_processor, dart_fundamental_processor, ohlcv_df
)

stock_info_df = stock_info_procesor.get_stock_info_df(stockcodes=stockcodes)
stock_info_df

,StockCode,StockName,MarketSector,Product,EPS,BPS,Price
0,035420,NAVER,KOSPI_서비스업,포털 서비스 및 온라인 광고,2706,94910,205500
1,221980,케이디켐,KOSDAQ_화학,PVC 안정제,1301,28567,12200
2,002450,삼익악기,KOSPI_기타제조업,"피아노,기타,현악기,전자악기,동 부분품 제조,임대,도매",21,3238,1091
3,044780,에이치케이,KOSDAQ_기계·장비,레이저가공 절단기,28,4370,1829
4,073560,우리손에프앤지,KOSDAQ_음식료·담배,"양돈업, 도축업",0,3048,1568
5,014190,원익큐브,KOSDAQ_유통,"MMA,BAM,PC,PP,목재,실란트,디지털인쇄기",37,4175,1824
6,007720,대명소노시즌,KOSDAQ_유통,기업소모성자재 구매대행업,9,1641,613
7,067900,와이엔텍,KOSDAQ_기타서비스,"사업장폐기물중간및최종처리,전문건설,골재제조",173,16897,7520


In [11]:
stockcodes = {'006260', '017390', '019680', '221980'}
stock_info_df = stock_info_procesor.get_stock_info_df(stockcodes=stockcodes)
stock_info_df

,StockCode,StockName,MarketSector,Product,EPS,BPS,Price
0,017390,서울가스,KOSPI_전기가스업,"도시가스공급,가스공급설비공사",-1635,296171,61100
1,006260,LS,KOSPI_기타금융,"전력용전선,일반전선,통신케이블,동선,알루미늄선,공조기,프라스틱사출성형기 제조,도매",1281,169868,85100
2,221980,케이디켐,KOSDAQ_화학,PVC 안정제,1301,28567,12200
3,019680,대교,KOSPI_서비스업,"학습지,참고서 출판/건설업/종합유선방송 프로그램공급,디지털학원",-115,7085,2570


In [12]:
stockcodes = {"006260", "017390", "221980"}